In [ ]:
%config Completer.use_jedi = False
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec

In [ ]:
eval_dir = '/home/flutz/PycharmProjects/social-force-model/output/'
sim_run = '20230104-151620-vci_front_01'

sim_eval_path = eval_dir + sim_run

In [ ]:
data_ped_path = '/home/flutz/PycharmProjects/social-force-model/evaluation/vci-dataset-citr-master/data/trajectories_filtered/vci_front/front_interaction_01_traj_ped_filtered.csv'
data_veh_path = '/home/flutz/PycharmProjects/social-force-model/evaluation/vci-dataset-citr-master/data/trajectories_filtered/vci_front/front_interaction_01_traj_veh_filtered.csv'

# Load data

Load data from dataset and from simulation and prepare data for comparison and evaluation.

## Dataset data

In [ ]:
#fps = 29.94
step_time = 0.0334

### Pedestrian data from dataset

In [ ]:
data_ped_df = pd.read_csv(data_ped_path)
data_ped_df.rename(columns={'id': 'ped_id', 'x_est': 'x', 'y_est': 'y', 'vx_est': 'v_x', 'vy_est': 'v_y'}, inplace=True)
data_ped_df['frame'] = data_ped_df['frame'] - data_ped_df['frame'].min()
data_ped_df['ped_id'] = data_ped_df['ped_id'] - data_ped_df['ped_id'].min()
data_ped_df['y'] = data_ped_df['y']*-1
data_ped_df['v_y'] = data_ped_df['v_y']*-1
data_ped_df.sort_values(by=['ped_id', 'frame'], inplace=True)
data_ped_df.insert(2, 'time', data_ped_df['frame'] * step_time)
data_ped_df.drop(columns=['label'], inplace=True)
data_ped_df['speed'] = np.linalg.norm(data_ped_df[['v_x', 'v_y']].to_numpy(), axis=-1)

data_offset_x = data_ped_df['x'].values[0]
data_offset_y = data_ped_df['y'].values[0]

data_ped_df['x'] = data_ped_df['x'] - data_offset_x
data_ped_df['y'] = data_ped_df['y'] - data_offset_y

data_ped_df

### Vehicle data from dataset

In [ ]:
veh_ext_x = 1.2
veh_ext_y = 0.6

In [ ]:
data_veh_df = pd.read_csv(data_veh_path)
data_veh_df.rename(columns={'id': 'veh_id', 'x_est': 'x', 'y_est': 'y', 'psi_est': 'heading', 'vel_est': 'vel'}, inplace=True)
data_veh_df['frame'] = data_veh_df['frame'] - data_veh_df['frame'].min()
data_veh_df['veh_id'] = data_veh_df['veh_id'] - data_veh_df['veh_id'].min()
data_veh_df['y'] = data_veh_df['y']*-1
data_veh_df['heading'] = data_veh_df['heading']*-1
data_veh_df.sort_values(by=['veh_id', 'frame'], inplace=True)
data_veh_df.insert(2, 'time', data_veh_df['frame'] * step_time)
data_veh_df['ext_x'] = veh_ext_x
data_veh_df['ext_y'] = veh_ext_y
data_veh_df.drop(columns=['label'], inplace=True)

data_veh_df['x'] = data_veh_df['x'] - data_offset_x
data_veh_df['y'] = data_veh_df['y'] - data_offset_y

data_veh_df

## Simulation data

In [ ]:
ped_time_offset = 1.5

### Pedestrian data from simulation

In [ ]:
sim_ped_df = pd.read_csv(sim_eval_path + '/pedestrian.csv')
sim_ped_df.sort_values(by=['ped_id', 'frame'], inplace=True)
sim_ped_df['y'] = sim_ped_df['y']*-1
sim_ped_df['v_y'] = sim_ped_df['v_y']*-1
sim_ped_df['speed'] = np.linalg.norm(sim_ped_df[['v_x', 'v_y']].to_numpy(), axis=-1)
#sim_ped_df = sim_ped_df[sim_ped_df.index < len(data_ped_df)]

sim_ped_df = sim_ped_df[sim_ped_df['time'] >= ped_time_offset]
sim_ped_df['frame'] = sim_ped_df['frame'] - sim_ped_df['frame'].min()
sim_ped_df['time'] = sim_ped_df['time'] - sim_ped_df['time'].min()

sim_offset_x = sim_ped_df['x'].values[0]
sim_offset_y = sim_ped_df['y'].values[0]

sim_ped_df['x'] = sim_ped_df['x'] - sim_offset_x
sim_ped_df['y'] = sim_ped_df['y'] - sim_offset_y

sim_ped_df

### Vehicle data from simulation

In [ ]:
sim_veh_df = pd.read_csv(sim_eval_path + '/vehicle.csv')
sim_veh_df['veh_id'] = sim_veh_df['veh_id'] - sim_veh_df['veh_id'].min()
sim_veh_df['y'] = sim_veh_df['y']*-1
sim_veh_df['heading'] = sim_veh_df['heading']*-1
sim_veh_df.sort_values(by=['veh_id', 'frame'], inplace=True)
sim_veh_df = sim_veh_df[sim_veh_df.index < len(data_veh_df)]

sim_veh_df['frame'] = sim_veh_df['frame'] - sim_veh_df['frame'].min()
sim_veh_df['time'] = sim_veh_df['time'] - sim_veh_df['time'].min()

sim_veh_df['x'] = sim_veh_df['x'] - sim_offset_x
sim_veh_df['y'] = sim_veh_df['y'] - sim_offset_y

sim_veh_df

# Plot

In [ ]:
def plot_scenario(ax, ped_df, veh_df, title, arrow_length=10):
    for key, grp in ped_df.groupby(['ped_id']):
        ax = grp.plot(ax=ax, kind='line', x='x', y='y', label=f'ped_{key}', legend=False)
        ax.arrow(grp['x'].iloc[0], grp['y'].iloc[0], grp['x'].iloc[arrow_length] - grp['x'].iloc[0], grp['y'].iloc[10] - grp['y'].iloc[0], width=0.1, zorder=10)

    for key, grp in veh_df.groupby(['veh_id']):
        ax = grp.plot(ax=ax, kind='line', ls='--', lw=2, x='x', y='y', label=f'veh_{key}', legend=False)
        ax.arrow(grp['x'].iloc[0], grp['y'].iloc[0], grp['x'].iloc[10] - grp['x'].iloc[0], grp['y'].iloc[10] - grp['y'].iloc[0], width=0.2, zorder=10)
    
    ax.set_title(title)
    ax.set_aspect('equal')
    ax.set_xlabel('x [m]')
    ax.set_ylabel('y [m]')
    ax.set_ylim([-6, 2])
    ax.set_xlim([-10, 25])


In [ ]:
fig, (ax1, ax2) = plt.subplots(2,1)

plot_scenario(ax1, data_ped_df, data_veh_df, 'Dataset')
plot_scenario(ax2, sim_ped_df, sim_veh_df, 'Simulation', arrow_length=10)

handles, labels = ax1.get_legend_handles_labels()
fig.legend(handles, labels, loc='center left', bbox_to_anchor=(1, 0.5))
fig.tight_layout()
#plt.savefig('/home/flutz/Schreibtisch/6_Months_Status_Presentation/vci_front_01_scenario_trajectory_comparison.png', dpi=300)
plt.show()

# Metrics

### Speed

In [ ]:
def average_speed(ped_df):
    for key, grp in ped_df.groupby(['ped_id']):
        speed = np.median(np.linalg.norm(grp[['v_x', 'v_y']].to_numpy(), axis=-1))
        print(f'Mean speed {key}: {speed}')
        
def plot_speeds(ax, ped_df, title):
    for key, grp in ped_df.groupby(['ped_id']):
        ax = grp.plot(ax=ax, kind='line', x='time', y='speed', label=f'ped_{key}', legend=False, ylim=[0,2])
        
        ax.set_title(title)
        ax.set_xlabel('time [s]')
        ax.set_ylabel('speed [m/s]') 
        
def plot_speed_errors(ax, sim_ped_df, data_ped_df):
    sim_groups = sim_ped_df.groupby(['ped_id'])

    for key, data_grp in data_ped_df.groupby(['ped_id']):

        sim_grp = sim_groups.get_group(key)

        data_speeds = data_grp[['speed']].to_numpy()
        sim_speeds = sim_grp[['speed']].to_numpy()
        
        max_len = min([len(data_speeds), len(sim_speeds)])
        data_speeds = data_speeds[: max_len]
        sim_speeds = sim_speeds[: max_len]
        
        loc_errors = np.linalg.norm(data_speeds - sim_speeds, axis=-1)

        ax.plot(data_grp['time'].to_numpy()[: max_len], loc_errors, label=f'ped_{key}')
        
        ax.set_xlabel('time [s]')
        ax.set_ylabel('speed error [m/s]')
        ax.set_title('Speed error over time')

In [ ]:
fig = plt.figure()
gs = GridSpec(2, 4, figure=fig)
ax1 = fig.add_subplot(gs[0, :2])
ax2 = fig.add_subplot(gs[1, :2], sharex=ax1)
ax3 = fig.add_subplot(gs[:, 2:4], sharex=ax1)

plot_speeds(ax1, data_ped_df, 'Dataset')
plot_speeds(ax2, sim_ped_df, 'Simulation')
plot_speed_errors(ax3, sim_ped_df, data_ped_df)

handles, labels = ax1.get_legend_handles_labels()
fig.legend(handles, labels, loc='center left', bbox_to_anchor=(1, 0.5))
fig.tight_layout()
plt.show()

### Location Error

In [ ]:
def plot_loc_errors(ax, sim_ped_df, data_ped_df):
    
    sim_groups = sim_ped_df.groupby(['ped_id'])

    for key, data_grp in data_ped_df.groupby(['ped_id']):

        sim_grp = sim_groups.get_group(key)

        data_locs = data_grp[['x', 'y']].to_numpy()
        sim_locs = sim_grp[['x', 'y']].to_numpy()
        
        max_len = min([len(data_locs), len(sim_locs)])
        data_locs = data_locs[: max_len]
        sim_locs = sim_locs[: max_len]
        
        loc_errors = np.linalg.norm(data_locs - sim_locs, axis=-1)

        ax = plt.plot(data_grp['time'].to_numpy()[: max_len], loc_errors, label=f'ped_{key}')
        
        

In [ ]:
fig, ax = plt.subplots()

plot_loc_errors(ax, sim_ped_df, data_ped_df)

handles, labels = ax.get_legend_handles_labels()
fig.legend(handles, labels, loc='center left', bbox_to_anchor=(1, 0.5))

ax.set_xlabel('time [s]')
ax.set_ylabel('location error [m]')
ax.set_title('Location error over time')

plt.show()
